In [3]:
file_path = "contracts/3.pdf"


In [2]:
import camelot
import pandas as pd
from find_cost import get_sort_csv  # Bu funksiyada text parametri bo'lishi kerak

def extract_clean_tables_from_pdf_with_camelot(pdf_path, output_prefix="jadval_clean"):
    # 1. Jadval(lar)ni PDF'dan o‘qish
    tables = camelot.read_pdf(pdf_path, pages='all', flavor='stream')

    for i, table in enumerate(tables):
        # 2. CSV fayl nomi
        raw_csv_file = f"{output_prefix}_{i+1}_raw.csv"

        # 3. CSV faylga saqlash (xom, tartiblanmagan holatda)
        table.to_csv(raw_csv_file)
        
        # 4. Faylni text formatda o‘qish
        with open(raw_csv_file, 'r', encoding='utf-8') as f:
            csv_text = f.read()

        # 5. get_sort_csv() funksiyasiga yuborish
        sorted_csv_text = get_sort_csv(csv_text)
        print(sorted_csv_text)
        # 6. Tartiblangan CSV ni xotiraga (diskka) yozish
        # sorted_csv_file = f"{output_prefix}_{i+1}_sorted.csv"
        # with open(sorted_csv_file, 'w', encoding='utf-8') as f:
        #     f.write(sorted_csv_text)
        
        break  # Faqat birinchi jadval bilan ishlash

# ✅ Misol chaqiruv:
extract_clean_tables_from_pdf_with_camelot(file_path)


ModuleNotFoundError: No module named 'find_cost'

In [12]:
import tabula
import pandas as pd

# 1. PDF fayl manzili
file_path = "contracts/4.pdf"

# 2. PDFdagi barcha sahifalardan jadval(lar)ni o‘qish
tables = tabula.read_pdf(file_path, pages='all', multiple_tables=True)

# 3. Har bir jadvalni alohida CSV faylga yozish
for i, table in enumerate(tables):
    csv_filename = f"jadval_{i+1}.csv"
    table.to_csv(csv_filename, index=False)
    print(f"✅ {csv_filename} faylga saqlandi.")


Got stderr: apr 25, 2025 5:48:27 TK org.apache.fontbox.ttf.OS2WindowsMetricsTable read
java.io.EOFException
	at org.apache.fontbox.ttf.TTFDataStream.readUnsignedInt(TTFDataStream.java:151)
	at org.apache.fontbox.ttf.OS2WindowsMetricsTable.read(OS2WindowsMetricsTable.java:843)
	at org.apache.fontbox.ttf.TrueTypeFont.readTable(TrueTypeFont.java:361)
	at org.apache.fontbox.ttf.TTFParser.parseTables(TTFParser.java:173)
	at org.apache.fontbox.ttf.TTFParser.parse(TTFParser.java:150)
	at org.apache.fontbox.ttf.OTFParser.parse(OTFParser.java:79)
	at org.apache.fontbox.ttf.OTFParser.parse(OTFParser.java:27)
	at org.apache.fontbox.ttf.TTFParser.parse(TTFParser.java:106)
	at org.apache.fontbox.ttf.OTFParser.parse(OTFParser.java:73)
	at org.apache.pdfbox.pdmodel.font.PDCIDFontType2.<init>(PDCIDFontType2.java:112)
	at org.apache.pdfbox.pdmodel.font.PDCIDFontType2.<init>(PDCIDFontType2.java:65)
	at org.apache.pdfbox.pdmodel.font.PDFontFactory.createDescendantFont(PDFontFactory.java:139)
	at org.apac

✅ jadval_1.csv faylga saqlandi.


In [9]:
import tabula
import pandas as pd

def extract_clean_tables_from_pdf(pdf_path, output_prefix="jadval_clean"):
    # 1. Jadval(lar)ni PDF'dan o‘qish
    tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True, guess=True)

    table_count = 0
    print(tables)

    for i, table in enumerate(tables):
        # 2. NaN tozalash (keraksiz ustun/qatordan qochish)
        if table.shape[1] < 4:
            continue  # juda oz ustunli jadval — tashlab ketamiz

        # 3. Ustun nomlarini tekshirish, bo‘sh bo‘lsa avtomatik nom berish
        if any(col.startswith("Unnamed") or col == "" for col in table.columns):
            table.columns = [f"Column_{j+1}" for j in range(table.shape[1])]

        # 4. Faqat 50% dan ko‘proq ustunlari to‘ldirilgan qatorlarni saqlaymiz
        threshold = int(table.shape[1] * 0.5)
        table_cleaned = table.dropna(thresh=threshold)

        if not table_cleaned.empty:
            table_count += 1
            output_name = f"{output_prefix}_{table_count}.csv"
            table_cleaned.to_csv(output_name, index=False)
            print(f"✅ {output_name} saqlandi")

    if table_count == 0:
        print("⚠️ Hech qanday toza jadval topilmadi.")
    else:
        print(f"🏁 {table_count} ta jadval saqlandi.")

# Misol chaqiruv
extract_clean_tables_from_pdf("contracts/7.pdf")


[   Unnamed: 0         Unnamed: 1                    Unnamed: 2 Unnamed: 3  \
0         NaN                NaN                           NaN        NaN   
1         NaN                NaN                           NaN        NaN   
2          No  Иш ва хизмат номи                 Улчов бирлиги       Сони   
3         NaN                NaN                           NaN        NaN   
4         NaN                NaN                           NaN        NaN   
5         NaN                NaN                           NaN        NaN   
6           1    Крупа гороховая         Единица измерения: кг       50.0   
7         NaN                NaN                           NaN        NaN   
8           2             Зелень         Единица измерения: кг       62.0   
9         NaN                NaN                           NaN        NaN   
10        NaN                NaN         Единица измерения: кг        NaN   
11        NaN                NaN  Наличие в составе сахара или        NaN  